Import libraries

In [24]:
import os 
import json
import numpy as np 

import torch
import torch.nn.functional as F
from utils import load_pickle

Get the results file

In [3]:
path = "/data/rbg/users/klingmin/projects/MS_processing/benchmarks/MIST/all_fp_preds.p"
predictions = load_pickle(path)

Helper Functions

In [35]:
def get_loss(FP_pred, FP):
    return F.binary_cross_entropy_with_logits(FP_pred, FP, reduce = False)

def batch_jaccard_index(FP_pred, FP):

    # Intersection = bitwise AND
    intersection = np.logical_and(FP, FP_pred).sum(axis=1)

    # Union = bitwise OR
    union = np.logical_or(FP, FP_pred).sum(axis=1)

    # Avoid division-by-zero by adding a small epsilon
    jaccard_scores = intersection / (union + 1e-9)
    total_jaccard = jaccard_scores.sum()

    return total_jaccard

Get rough estimation of scores

In [15]:
pred_0, target_0 = predictions[0]["preds"], predictions[0]["targs"]
pred_1, target_1 = predictions[1]["preds"], predictions[1]["targs"]

In [38]:
total_loss, total_jaccard, count = 0, 0, 0 

for p, t in zip(pred_0, target_0):

    p = torch.from_numpy(np.array(p)).unsqueeze(0).float()
    t = torch.from_numpy(np.array(t)).unsqueeze(0).float()

    score = get_loss(p, t).mean()
    jaccard = batch_jaccard_index(p, t)
    
    total_jaccard += jaccard
    total_loss += score 
    count += 1 

avg_jaccard = total_jaccard / count
avg_loss = total_loss / count
print(avg_loss, avg_jaccard)

/data/rbg/users/klingmin/anaconda3/envs/chem/lib/python3.12/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


tensor(0.6979) tensor(0.0229)


In [39]:
total_loss, total_jaccard, count = 0, 0, 0 

for p, t in zip(pred_1, target_1):

    p = torch.from_numpy(np.array(p)).unsqueeze(0).float()
    t = torch.from_numpy(np.array(t)).unsqueeze(0).float()

    score = get_loss(p, t).mean()
    jaccard = batch_jaccard_index(p, t)
    
    total_jaccard += jaccard
    total_loss += score 
    count += 1 

avg_jaccard = total_jaccard / count
avg_loss = total_loss / count
print(avg_loss, avg_jaccard)

/data/rbg/users/klingmin/anaconda3/envs/chem/lib/python3.12/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


tensor(0.6960) tensor(0.0229)
